In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import butcher
import bro
import os

from astropy.io import ascii
from astropy.timeseries import LombScargle

In [ ]:
#Reading in the data

#Get the data directory
cwd = os.getcwd()
data_dir = cwd.replace('Figure_4', 'Data\\')

#ASAS data
orgasas_data = ascii.read(data_dir + 'asas.csv')
asas_mask = (orgasas_data['emag'] < 0.05)
asas_data = orgasas_data[asas_mask]

asas_flux = butcher.mag_to_flux(asas_data['mag'])
asas_eflux = butcher.emag_to_eflux(asas_data['mag'], asas_data['emag'])

#ASASSN data
orgasassn_data = ascii.read(data_dir + 'asassn.csv')
asassn_mask = (orgasassn_data['emag'] < 0.05)
asassn_data = orgasassn_data[asassn_mask]

asassn_flux = butcher.mag_to_flux(asassn_data['mag'])
asassn_eflux = butcher.emag_to_eflux(asassn_data['mag'], asassn_data['emag'])

#KELT data
orgkelt_data = ascii.read(data_dir + 'kelt.csv')
kelt_mask = (orgkelt_data['emag'] < 0.05)
kelt_data = orgkelt_data[kelt_mask]

kelt_flux = butcher.mag_to_flux(kelt_data['mag'])
kelt_eflux = butcher.emag_to_eflux(kelt_data['mag'], kelt_data['emag'])

#PROMPT data
orgprompt_data = ascii.read(data_dir + 'prompt.csv') #time is JD-2450000
prompt_mask = (orgprompt_data['emag'] < 0.05)
prompt_data = orgprompt_data[prompt_mask]

prompt_flux = butcher.mag_to_flux(prompt_data['mag'])
prompt_eflux = butcher.emag_to_eflux(prompt_data['mag'], prompt_data['emag'])

#ROAD data
orgroad_data = ascii.read(data_dir + 'road.csv') #time is JD-2450000
road_mask = (orgroad_data['emag'] < 0.05)
road_data = orgroad_data[road_mask]

road_flux = butcher.mag_to_flux(road_data['mag'])
road_eflux = butcher.emag_to_eflux(road_data['mag'], road_data['emag'])

In [ ]:
#Correct for the long term flux decrease mentioned in section 3.1

asas_flux = butcher.long_correct(asas_data['MJD'], asas_flux, asas_eflux)
asassn_flux = butcher.long_correct(asassn_data['MJD'], asassn_flux, asassn_eflux)
kelt_flux = butcher.long_correct(kelt_data['HJD'], kelt_flux, kelt_eflux)
prompt_flux = butcher.long_correct(prompt_data['HJD'], prompt_flux, prompt_eflux)
road_flux = butcher.long_correct(road_data['HJD'], road_flux, road_eflux)

In [ ]:
#Store the individual telescopes in lists

times = [asas_data['MJD'], asassn_data['MJD'], kelt_data['HJD'], prompt_data['HJD'], road_data['HJD']]
fluxes = [asas_flux, asassn_flux, kelt_flux, prompt_flux, road_flux]
uncertainties = [asas_eflux, asassn_eflux, kelt_eflux, prompt_eflux, road_eflux]
names = ['ASAS', 'ASAS-SN', 'KELT', 'PROMPT', 'ROAD']

In [ ]:
#Remove the periodicities

names = ['ASAS', 'ASASSN', 'KELT', 'PROMPT', 'ROAD']
org_powers, powers2 = [], []

for j in range(5):
    time, flux, eflux = times[j], fluxes[j], uncertainties[j]

    corrflux1, periods1, freq1, power1 = bro.short_correct(time, flux, eflux, min_chunk_size = 10)

    #Get the uncorrected lombscargle
    frequencies = 1/np.linspace(2, 10, 3000)
    org_power = LombScargle(time, flux-np.mean(flux), dy = eflux).power(frequencies)
    
    #Get the doubly bro corrected lombscargle
    corrflux2, periods2, freq2, power2 = bro.short_correct(time, corrflux1, eflux, min_chunk_size = 10)
    frequencies = 1/np.linspace(2, 10, 3000)
    power2 = LombScargle(time, corrflux2-np.mean(corrflux2), dy = eflux).power(frequencies)
    
    org_powers.append(org_power)
    powers2.append(power2)

In [ ]:
#Create the figure 

import matplotlib
plt.style.use('seaborn-dark-palette')
font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 20}
matplotlib.rc('font', **font)

fig, ax = plt.subplots(5)
for i in range(5):
    ax[i].plot(1/frequencies, org_powers[i], alpha = 0.7, c='grey', label = 'Pre-Correction ' + names[i])
    ax[i].plot(1/frequencies, powers2[i], alpha = 0.7,label = 'Post-Correction ' + names[i])
    ax[i].legend(fontsize = 18)

fig.text(0.35, 0.09, 'Signal Period (Days)', fontsize = 24)
fig.text(0.02, 0.35, 'Signal Power (Arbitrary Units)', rotation = 90, fontsize = 24)
fig = plt.gcf()
fig.set_size_inches(12,20)
#plt.savefig('Removing_Dominant_Cycle.pdf')
plt.show()